In [ ]:
import pandas as pd
from pyprojroot import here

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
data['Hospital Name'].unique().shape

In [ ]:
data = data.iloc[:,[22, 12, 20]]

In [ ]:
data.columns = ['country', 'position', 'hospital_type']

In [ ]:
conversion = {
    'Central or Regional Hospital (Secondary Care)' : 'Secondary',
    'Local or Community Hospital (Primary to Secondary Care)' : 'Primary',
    'Specialized hospital (e.g. cardiology, orthopedics or psychiatry)': 'Tertiary',
    'University hospital (Tertiary Care)' : 'Tertiary'}

In [ ]:
data.hospital_type = data.hospital_type.replace(conversion)

In [ ]:
country = data.groupby('country').position.value_counts().unstack().fillna(0).T
hospital_type = data.groupby('country').hospital_type.value_counts().unstack().fillna(0).T
table = pd.concat([country, hospital_type], keys=['Position', 'Hospital'])

In [ ]:
# Define the exact order you want
desired_order = [
    ('Position', 'Other'),
    ('Position', 'Trainee'),
    ('Position', 'Resident'),
    ('Position', 'Specialist'),
    ('Position', 'Head'),
    ('Hospital', 'Other'),
    ('Hospital', 'Primary'),
    ('Hospital', 'Secondary'),
    ('Hospital', 'Tertiary')
]

# Reindex to this order (will only include rows that exist)
table = table.reindex(desired_order)

# Calculate the sum of all Position rows for each country
position_total = table.loc['Position'].sum(axis=0)
# Add as a new row with empty first level index
table.loc[('', 'Total'), :] = position_total

In [ ]:
table['Total'] = table.sum(axis=1)

In [ ]:
column_format = 'p{1.0cm}' + 'p{1.1cm}' + 'p{0.07cm}'*table.shape[1]

In [ ]:
table.columns = [f"\\rotatebox{{90}}{{{x}}}" for x in table.columns]

In [ ]:
filepath = here() / 'output/tables/description.tex'

table.to_latex(
    buf=filepath,
    float_format='%.0f',
    column_format=column_format,
    label='tab:description',
    caption='Description',
    position='H'
)

```
\makebox[\textwidth][c]{
    \resizebox{1.2\textwidth}{!}{%  <--- allow controlled overflow
    }}
```

In [ ]:
# Replace 'your_file.tex' with your actual filename
filename = filepath

# Read all lines
with open(filename, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Insert formatting commands after caption/label (around line 3)
if len(lines) >= 3:
    lines.insert(3, "\\footnotesize\n")
    lines.insert(4, "\\makebox[\\textwidth][c]{%\n")
    lines.insert(5, "\\resizebox{1.2\\textwidth}{!}{%\n")

# Find the line with \end{tabular} and insert }} after it
for i, line in enumerate(lines):
    if "\\end{tabular}" in line:
        lines.insert(i + 1, "}}\n")
        break

# Write the modified lines back to the file
with open(filename, "w", encoding="utf-8") as f:
    f.writelines(lines)

print("LaTeX formatting updated successfully!")